# COVID-19 en Chile
> Total de casos confirmados, fallecidos confirmados, pacientes en UCI por región.

- toc: true 
- badges: true
- comments: true
- author: Alonso Silva Allende
- categories: [jupyter]
- image: images/chart-preview.png

In [1]:
#hide
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
plt.style.use('ggplot')

In [2]:
#hide
from IPython.display import display_html, HTML

In [3]:
#hide
update_date = pd.to_datetime('today') - pd.offsets.Hour(19)
today = update_date.strftime('%Y-%m-%d')
today

'2020-04-24'

In [5]:
#hide
date_one_week_ago = (update_date - pd.offsets.Day(7)).strftime('%Y-%m-%d')
date_one_week_ago

'2020-04-17'

In [6]:
#hide_input
print(f"Última actualización: {update_date.strftime('%d/%m/%Y')}.")

Última actualización: 24/04/2020.


In [7]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [8]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ today + "-CasosConfirmados-totalRegional.csv"
deaths_raw = pd.read_csv(s, index_col='Region')

In [9]:
#hide
s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" \
+ date_one_week_ago + "-CasosConfirmados-totalRegional.csv"
deaths_one_week_ago = pd.read_csv(s, index_col='Region')

In [10]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')

In [11]:
#hide
data_table = pd.DataFrame()
data_table["Region"] = casos_raw[today].drop("Total").reset_index()["Region"]
data_table["CC"] = casos_raw[today].drop("Total").reset_index()[today]
data_table["CC por 100.000 hab."] = np.round(100000*(casos_raw[today]/tests_raw['Poblacion']).drop('Total'), decimals=1).values
data_table["FC"] = deaths_raw["Fallecidos"].drop("Total").values
data_table["FC por 100.000 hab."] = \
np.round(100000*deaths_raw['Fallecidos'].drop('Total')/(tests_raw['Poblacion'].values), decimals=1).values

In [12]:
#hide
tests_raw = tests_raw.fillna(0)

In [13]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [14]:
#hide
tests_raw.loc['Total'] = tests_total

In [15]:
#hide
tests_table = pd.DataFrame()
tests_table["Región"] = tests_raw.reset_index()['Region']
tests_table["Exámenes informados"] = tests_raw.drop(columns='Poblacion').sum(axis=1).values
tests_table["Exámenes informados por 100.000 habs."] = \
np.round(100000*tests_raw.drop(columns='Poblacion').sum(axis=1).values/tests_raw['Poblacion'], decimals=1).values

In [16]:
#hide
total_confirmed = casos_raw.iloc[-1,-1]
total_deaths = deaths_raw.iloc[-1,-1]

In [17]:
#hide
total_confirmed_one_week_ago = casos_raw.iloc[-1,-8]
total_deaths_one_week_ago = deaths_one_week_ago.loc["Total", "Fallecidos"]

In [18]:
#hide
diff_cases = total_confirmed-total_confirmed_one_week_ago
diff_deaths = total_deaths-total_deaths_one_week_ago

In [19]:
#hide
total_confirmed = '{:,}'.format(total_confirmed).replace(',', '.')
total_deaths = '{:,}'.format(total_deaths).replace(',', '.')

In [20]:
#hide
diff_cases = '{:,}'.format(diff_cases).replace(',', '.')
diff_deaths = '{:,}'.format(diff_deaths).replace(',', '.')

# Estadísticas para Chile

In [21]:
#hide_input
HTML(f'<p style="color:#3361ff;"><span style="font-weight:bold;">Casos confirmados:</span> {total_confirmed} (+{diff_cases} en una semana)</p><p style="color:#FF3F3F;"><span style="font-weight:bold;">Fallecimientos confirmados:</span> {total_deaths} (+{diff_deaths} en una semana)</p>')

In [22]:
#hide_input
(data_table.style.set_properties(**{'text-align': 'right'}).background_gradient(cmap='Reds').hide_index()).set_caption('Estadísticas por región: Casos confirmados (CC) y fallecidos confirmados (FC)')

Region,CC,CC por 100.000 hab.,FC,FC por 100.000 hab.
Arica y Parinacota,239,52.9,3,1.2
Tarapacá,134,116.7,0,0
Antofagasta,366,94.8,2,0.3
Atacama,21,6.7,0,0
Coquimbo,72,6.5,0,0
Valparaíso,436,41.1,8,0.4
Metropolitana,6761,8.6,86,1.1
O’Higgins,81,51.6,1,0.1
Maule,349,42.9,10,0.9
Ñuble,715,349.9,14,2.7


In [23]:
#hide_input
(tests_table[:-1].style.set_properties(**{'text-align': 'right'}).background_gradient(cmap='Blues').hide_index()).set_caption('Estadísticas por región: Exámenes informados desde el 9 de abril')

Región,Exámenes informados,Exámenes informados por 100.000 habs.
Arica y Parinacota,1143,453.4
Tarapacá,413,107.9
Antofagasta,2306,333.3
Atacama,658,209.1
Coquimbo,99,11.8
Valparaíso,3844,196.1
Metropolitana,52372,644.6
O’Higgins,685,69.1
Maule,4758,420.3
Ñuble,1997,390.4


Nota: Los exámenes informados por región empezaron a informarse sólo desde el 9 de abril.

# Evolución de casos confirmados por región

In [45]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [46]:
#hide
data_raw = data_raw.drop("Total")

In [47]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [48]:
#hide
data_raw = data_raw.reset_index()
regiones = data_raw['Region'].values
data = data.T
data = data.rename(columns={0: "date", 1: "region", 2: "casos", 3: "codigo region"})

In [49]:
#hide
data["casos"] = data["casos"].astype(int)

In [50]:
#hide_input
input_dropdown = alt.binding_select(options=data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', legend=None),
                    alt.value('lightgray'))

chart = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('casos', axis=alt.Axis(title='Casos confirmados')),
    color=color,
    tooltip=['date', 'region', 'casos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
).properties(
    title='COVID-19 en Chile: Total de casos confirmados por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

chart.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

Inspirado del [tweet](https://twitter.com/NachoToledoR/status/1244631395781218306) de [@NachoToledoR](https://twitter.com/NachoToledoR).

# Evolución de fallecidos confirmados por región

In [32]:
#hide
data = pd.DataFrame()
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-03-24-CasosConfirmados-totalRegional.csv",
    index_col='Region')
data['2020-03-24'] = data_raw['Fallecidos']

In [33]:
#hide
first_death_date = '2020-03-24'
total_days = (pd.to_datetime(today)-pd.to_datetime(first_death_date)).days

In [34]:
#hide
for i in np.arange(total_days+1):
  date = (pd.to_datetime(first_death_date)+pd.DateOffset(i)).strftime('%Y-%m-%d')
  s = "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/" + date + "-CasosConfirmados-totalRegional.csv"
  data_by_date = pd.read_csv(s)
  if 'Fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Fallecidos"].values
  elif 'Casos fallecidos' in data_by_date.columns:
    data[date] = data_by_date["Casos fallecidos"].values
  else:
    data[date] = data_by_date[" Casos fallecidos"].values

In [35]:
#hide
data = data.drop("Total")

In [36]:
#hide
new_data = pd.DataFrame()
i = 0
for date in data.keys():
    for n, region in enumerate(data.index):
        new_data[i] = date, region, data[date].loc[region], n
        i += 1

In [37]:
#hide
data = data.reset_index()
regiones = data['Region'].values
new_data = new_data.T
new_data = new_data.rename(columns={0: "date", 1: "region", 2: "fallecidos", 3: "codigo region"})

In [38]:
#hide
new_data["fallecidos"] = new_data["fallecidos"].astype(int)

In [43]:
#hide_input

input_dropdown = alt.binding_select(options=new_data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(new_data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('fallecidos', axis=alt.Axis(title='Fallecidos confirmados')),
    color=color,
    tooltip=['date', 'region', 'fallecidos'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title='COVID-19 en Chile: Total de fallecidos confirmados por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Evolución de pacientes en UCI por región

In [51]:
#hide
data_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto8/UCI.csv',
    index_col='Region')
data_raw = data_raw.drop(columns='Codigo region')
data_raw.loc['Total'] = data_raw.sum(axis=0)
data_raw = data_raw.drop(columns='Poblacion')

In [52]:
#hide
data_raw = data_raw.drop("Total")

In [53]:
#hide
data = pd.DataFrame()
i = 0
for date in data_raw.keys():
    for n, region in enumerate(data_raw.index):
        data[i] = date, region, data_raw[date].loc[region], n
        i += 1

In [54]:
#hide
data_raw = data_raw.reset_index()

In [55]:
#hide
regiones = data_raw['Region'].values

In [56]:
#hide
data = data.T

In [57]:
#hide
data = data.rename(columns={0: "date", 1: "region", 2: "pacientes", 3: "codigo region"})

In [58]:
#hide
data["pacientes"] = data["pacientes"].astype(int)

In [60]:
#hide_input
input_dropdown = alt.binding_select(options=data['region'].unique())
selection1 = alt.selection_single(fields=['region'], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=['region'], on='mouseover')

color = alt.condition(selection1 | selection2,
                    alt.Color('region:N', legend=None),
                    alt.value('lightgray'))

bars = alt.Chart(data).mark_bar().encode(
    x=alt.X('date:O', axis=alt.Axis(title='Fecha (año-mes-día)')),
    y=alt.Y('pacientes'),
    color=color,
    tooltip=['date', 'region', 'pacientes'],
    order=alt.Order(
    # Sort the segments of the bars by this field
    'codigo region',
    sort='descending'
    )
    ).properties(
    title=f'COVID-19 en Chile: Pacientes en UCI por región'
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

legend = alt.Chart(new_data).mark_point().encode(
    y=alt.Y('region:N', axis=alt.Axis(orient='right'), sort=regiones),
    color=color
).add_selection(
    selection1, selection2
)

bars.properties(width=600, height=400) | legend

alt.HConcatChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Casos confirmados por región

In [61]:
#hide
data_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv",
    index_col='Region')

In [62]:
#hide
data_raw = data_raw.drop('Total')
data_raw = data_raw.reset_index()

In [63]:
#hide
regiones = data_raw["Region"].values
regiones

array(['Arica y Parinacota', 'Tarapacá', 'Antofagasta', 'Atacama',
       'Coquimbo', 'Valparaíso', 'Metropolitana', 'O’Higgins', 'Maule',
       'Ñuble', 'Biobío', 'Araucanía', 'Los Ríos', 'Los Lagos', 'Aysén',
       'Magallanes'], dtype=object)

In [64]:
#hide
data_raw = data_raw[['Region', today]]

In [65]:
#hide
today_es_format = update_date.strftime('%d/%m/%Y')

In [66]:
#hide_input
bars = alt.Chart(data_raw).mark_bar().encode(
#     x = alt.X(today, scale=alt.Scale(domain=(0, 6500)), axis=alt.Axis(title='Casos confirmados')),
    x = alt.X(today, axis=alt.Axis(title='Casos confirmados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = today 
).properties(
    title=f'COVID-19 en Chile: Casos confirmados por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=today
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Casos confirmados por región por cada 100.000 habitantes

In [67]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')

In [68]:
#hide
data_raw["Tasa de casos"] = 100000*data_raw[today]/tests_raw["Poblacion"].values

In [69]:
#hide_input

bars = alt.Chart(data_raw).mark_bar().encode(
    x = alt.X('Tasa de casos:Q', scale=alt.Scale(domain=(0, 340)), axis=alt.Axis(title='Casos confirmados por cada 100.000 habitantes')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'Tasa de casos'
).properties(
    title=f'COVID-19 en Chile: Casos confirmados por región por cada 100.000 habitantes al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de casos:Q', format=',.2f')
)

# bars.configure_title(
#     fontSize=20,
#     font='Courier',
#     anchor='start',
#     color='gray'
# )

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Fallecidos confirmados por región

In [70]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today +'-CasosConfirmados-totalRegional.csv'
data_raw = pd.read_csv(s, index_col='Region')

In [71]:
#hide
data_raw = data_raw[data_raw['Fallecidos']>0]
data_raw = data_raw.drop('Total')

In [72]:
#hide_input
bars = alt.Chart(data_raw.reset_index()).mark_bar().encode(
    x = alt.X('Fallecidos:Q', axis=alt.Axis(title='Fallecidos confirmados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'Fallecidos'
).properties(
    title=f'COVID-19 en Chile: Fallecidos confirmados por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='Fallecidos:Q'
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Fallecidos confirmados por región por cada 100.000 habitantes

In [73]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today + '-CasosConfirmados-totalRegional.csv'
data_raw = pd.read_csv(s, index_col='Region')

In [74]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')
tests_raw = tests_raw.fillna(0)

In [75]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [76]:
#hide
tests_raw.loc['Total'] = tests_total

In [77]:
#hide
death_rate = 100000*data_raw['Fallecidos']/tests_raw['Poblacion'].values
death_rate = death_rate.sort_values(ascending=True)
death_rate = death_rate[death_rate>0]

In [78]:
#hide_input
bars = alt.Chart(death_rate.reset_index()).mark_bar().encode(
    x = alt.X('Fallecidos:Q', axis=alt.Axis(title='Fallecidos confirmados por cada 100.000 habitantes')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'Fallecidos'
).properties(
    title=f'COVID-19 en Chile: Fallecidos confirmados por región por cada 100.000 habitantes al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Fallecidos:Q', format=',.2f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Tasa de letalidad por región
> Tasa de letalidad = fallecidos confirmados / casos confirmados (en porcentaje)

In [79]:
#hide
CFR = data_raw['Fallecidos']/data_raw['Casos  totales']*100
CFR = CFR.sort_values(ascending=True)
CFR = CFR[CFR>0]

In [80]:
#hide
CFR = CFR.reset_index()
CFR['Tasa de letalidad'] = CFR[0]
CFR = CFR.drop(columns=0)

In [81]:
#hide_input
bars = alt.Chart(CFR).mark_bar().encode(
    x = alt.X('Tasa de letalidad:Q', axis=alt.Axis(title='Tasa de letalidad')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'Tasa de letalidad'
).properties(
    title=f'COVID-19 en Chile: Tasa de letalidad por región al {today_es_format}'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('Tasa de letalidad:Q', format=',.1f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes informados por región desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

In [82]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv'
data_20200408 = pd.read_csv(s, index_col='Region')

In [83]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today +'-CasosConfirmados-totalRegional.csv'
data_update_date = pd.read_csv(s, index_col='Region')

In [84]:
#hide
tests_raw = pd.read_csv(
    'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto7/PCR.csv',
    index_col='Region')
tests_raw = tests_raw.fillna(0)

In [85]:
#hide
tests_total = pd.Series(dtype='float64')
for date in tests_raw.columns:
  tests_raw[date] = tests_raw[date].apply(lambda x: 0 if x=='-' else x)
  tests_raw[date] = tests_raw[date].astype(int)
  tests_total[date] = np.sum(tests_raw[date])

In [86]:
#hide
tests_raw.loc['Total'] = tests_total

In [87]:
#hide
data = pd.DataFrame()
data["Fallecidos"] = data_update_date["Fallecidos"]-data_20200408["Fallecidos"]
data["Casos"] = data_update_date["Casos  totales"]-data_20200408["Casos  totales"]
data["Tests"] = tests_raw.drop(columns='Poblacion').sum(axis=1).values

In [88]:
#hide
data_per_capita = pd.DataFrame()
data_per_capita["Fallecidos"] = 100000*data["Fallecidos"]/tests_raw["Poblacion"].values
data_per_capita["Casos"] = 100000*data["Casos"]/tests_raw["Poblacion"].values
data_per_capita["Tests"] = 100000*data["Tests"]/tests_raw["Poblacion"].values

In [89]:
#hide
casos_raw = pd.read_csv(
    "https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto3/CasosTotalesCumulativo.csv", 
    index_col='Region')

In [90]:
#hide
casos_20200408 = pd.read_csv('https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/2020-04-08-CasosConfirmados-totalRegional.csv', index_col='Region')

In [91]:
#hide
s = 'https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto4/' + today + '-CasosConfirmados-totalRegional.csv'
casos_update_date = pd.read_csv(s, index_col='Region')

In [92]:
#hide
positividad = 100*(casos_update_date['Casos  totales'] - casos_20200408['Casos  totales'])/tests_raw.drop(columns='Poblacion').sum(axis=1).values

In [93]:
#hide
tests_raw['tests informados'] = tests_raw.drop(columns='Poblacion').sum(axis=1)

In [94]:
#hide
tests_raw['tasa de tests'] = 100000*tests_raw['tests informados']/tests_raw['Poblacion']

In [95]:
#hide_input
bars = alt.Chart(tests_raw.drop('Total').reset_index()).mark_bar().encode(
    x = alt.X('tests informados:Q', axis=alt.Axis(title='Exámenes informados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'tests informados'
).properties(
    title='COVID-19 en Chile: Total de exámenes informados por región desde el 9 de abril'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('tests informados:Q', format='.0f')
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes informados por región por cada 100.000 habitantes desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

In [96]:
#hide_input
bars = alt.Chart(tests_raw.drop('Total').reset_index()).mark_bar().encode(
    x = alt.X('tasa de tests:Q', axis=alt.Axis(title='Exámenes informados')),
    y = alt.Y('Region:N', sort='-x'),
    tooltip = 'tasa de tests'
).properties(
    title='COVID-19 en Chile: Exámenes informados por región por cada 100.000 habitantes desde el 9 de abril'
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text=alt.Text('tasa de tests:Q', format='.1f')
)

bars.configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

(bars + text).properties(width=600, height=300)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por región  desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por región desde el 9 de abril.

In [97]:
#hide
data = data.sort_values(by="Tests", ascending=False)
data_per_capita = data_per_capita.sort_values(by="Tests", ascending=False)

In [98]:
#hide
data["codigo"]=np.arange(len(data))
data_per_capita["codigo"]=np.arange(len(data))

In [100]:
#hide_input
bar = alt.Chart(data.drop("Total").reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados')),
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos'],
).properties(
    title='COVID-19 en Chile: Exámenes informados, casos confirmados y fallecidos confirmados por región',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.55,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending'))
)

tick1 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.55,  # controls width of tick.
).encode(
    x='Casos:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Casos'
)

tick2 = alt.Chart(data.drop("Total").reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.55,  # controls width of tick.
).encode(
    x='Fallecidos:Q',
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Fallecidos'
)

(bar + tick0 + tick1 + tick2).properties(width=600, height=400)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por región por cada 100.000 habitantes desde el 9 de abril
Nota: sólo se dispone de los datos de exámenes informados por región desde el 9 de abril

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por región por cada 100.000 habitantes desde el 9 de abril

In [101]:
#hide_input
bar = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados')),
    y=alt.Y('Region:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['Region', 'Tests', 'Casos', 'Fallecidos'],
).properties(
    title='COVID-19 en Chile: Exámenes informados, casos confirmados y fallecidos confirmados por región por cada 100.000 habitantes',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.55,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Tests'
)

tick1 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.55,  # controls width of tick.
).encode(
    x='Casos:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Casos'
)

tick2 = alt.Chart(data_per_capita.drop("Total").reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.55,  # controls width of tick.
).encode(
    x='Fallecidos:Q',
    y=alt.Y('Region', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Fallecidos'
)

(bar + tick0 + tick1 + tick2).properties(width=600, height=400)

alt.LayerChart(...)

Fuente: [Ministerio de Ciencia](https://github.com/MinCiencia/Datos-COVID19)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de la OCDE por cada 1 millón de habitantes

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

In [102]:
#hide
test_data_raw = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/testing/covid-testing-all-observations.csv')

In [103]:
#hide
list_OCDE = ['Australia', 'Austria', 'Belgium', 'Canada', 'Chile', 
                  'Czech Republic', 'Denmark', 'Estonia', 'Finland', 'France',
                  'Germany', 'Greece', 'Hungary', 'Iceland', 'Ireland',
                  'Israel', 'Italy', 'Japan', 'Korea', 'Latvia',
                  'Lithuania', 'Luxembourg', 'Mexico', 'Netherlands', 'New Zealand',
                  'Norway', 'Poland', 'Portugal', 'Slovakia', 'Slovenia',
                  'Spain', 'Sweden', 'Switzerland', 'Turkey', 'United Kingdom']

In [104]:
#hide
test_data = pd.DataFrame()
for country in list_OCDE:
    test_data[country] = test_data_raw[test_data_raw['Entity'].str.contains(country)].iloc[-1]

In [105]:
#hide
# United States tiene dos series. Utilizo la que tiene datos más recientes.
test_data['United States'] = test_data_raw[test_data_raw['Entity'].str.contains('United States - inconsistent units')].iloc[-1]

In [106]:
#hide
test_data = test_data.T

In [107]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['Cumulative total']

In [108]:
#hide
list_OCDE = list_OCDE + ['United States']

In [109]:
#hide
full_data = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/full_data.csv", index_col='date')

In [110]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_OCDE:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_cases']
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_deaths']

In [111]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [112]:
#hide
locations_data = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/locations.csv')

In [113]:
#hide
population = pd.Series()
for country in list_OCDE:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [114]:
#hide
data['Population'] = population

In [115]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [116]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [118]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths'],
).properties(
    title='COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Tests'
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Cases'
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Deaths'
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)

Fuentes: ECDC, [Our World in Data](https://ourworldindata.org/)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de la OCDE por cada 1 millón de habitantes en los últimos 3 días

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

In [119]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['3-day rolling mean daily change']

In [120]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_OCDE:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_cases'].mean()
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_deaths'].mean()

In [121]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [122]:
#hide
population = pd.Series()
for country in list_OCDE:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [123]:
#hide
data['Population'] = population

In [124]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [125]:
#hide
data = data.dropna()

In [126]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [128]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title=f'Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes\n(promedio de los últimos 3 días)')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths'],
).properties(
    title=f'COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes\n(promedio de los últimos 3 días)',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Tests'
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Cases'
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = 'Deaths'
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de latinoamérica por cada 1 millón de habitantes

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

Nota: Muchos países de latinoamérica no tienen datos disponibles.

In [129]:
#hide
list_latinoamerica = ['Argentina', 'Bolivia', 'Chile', 'Colombia',
                     'Costa Rica', 'Cuba', 'Ecuador', 'El Salvador',
                     'Mexico', 'Panama', 'Paraguay',
                     'Peru', 'Uruguay']

In [130]:
#hide
test_data = pd.DataFrame()
for country in list_latinoamerica:
    test_data[country] = test_data_raw[test_data_raw['Entity'].str.contains(country)].iloc[-1]

In [131]:
#hide
test_data = test_data.T

In [132]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['Cumulative total']

In [133]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_latinoamerica:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_cases']
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-1]['total_deaths']

In [134]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [135]:
#hide
population = pd.Series()
for country in list_latinoamerica:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [136]:
#hide
data['Population'] = population

In [137]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [138]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [142]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title='Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
).properties(
    title='COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)

Fuentes: ECDC, [Our World in Data](https://ourworldindata.org/)

# Exámenes, casos confirmados, y fallecimientos confirmados por país de latinoamérica por cada 1 millón de habitantes en los últimos 3 días

> <span style="color:green">Exámenes</span>, <span style="color:blue">casos confirmados</span>, y <span style="color:red">fallecimientos confirmados</span> por país por cada 1 millón de habitantes

Nota: Muchos países de latinoamérica no tienen datos disponibles.

In [143]:
#hide
data = pd.DataFrame()
data['Tests'] = test_data['3-day rolling mean daily change']

In [144]:
#hide
cases = pd.Series()
deaths = pd.Series()
for country in list_latinoamerica:
    cases[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_cases'].mean()
    deaths[country] = full_data[full_data['location'].str.contains(country)].iloc[-3:]['new_deaths'].mean()

In [145]:
#hide
data['Cases'] = cases
data['Deaths'] = deaths

In [146]:
#hide
population = pd.Series()
for country in list_latinoamerica:
    population[country] = locations_data[locations_data['location'].str.contains(country)].iloc[-1]['population']

In [147]:
#hide
data['Population'] = population

In [148]:
#hide
data['Tests'] = 1000000*data['Tests']/data['Population']
data['Cases'] = 1000000*data['Cases']/data['Population']
data['Deaths'] = 1000000*data['Deaths']/data['Population']

In [149]:
#hide
data = data.dropna()

In [150]:
#hide
data = data.sort_values(by='Tests', ascending=False)
data['codigo'] = np.arange(len(data))

In [155]:
#hide_input
bar = alt.Chart(data.reset_index()).mark_bar(opacity=0.4, color='green').encode(
    x=alt.X('Tests:Q', axis=alt.Axis(title=f'Exámenes informados, casos confirmados, fallecimientos confirmados por cada 1 millón de habitantes\n(promedio de los últimos 3 días)')),
    y=alt.Y('index:N', axis=alt.Axis(title='País'), sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
).properties(
    title=f'COVID-19: Exámenes informados, casos confirmados y fallecidos confirmados por país por cada 1 millón de habitantes\n(promedio de los últimos 3 días)',
    width=alt.Step(40)  # controls width of bar.
)

tick0 = alt.Chart(data.reset_index()).mark_tick(
    color='black',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Tests:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick1 = alt.Chart(data.reset_index()).mark_tick(
    color='blue',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Cases:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

tick2 = alt.Chart(data.reset_index()).mark_tick(
    color='red',
    thickness=2,
    size=40 * 0.43,  # controls width of tick.
).encode(
    x='Deaths:Q',
    y=alt.Y('index:N', sort=alt.EncodingSortField(field='codigo', order='ascending')),
    tooltip = ['index', 'Tests', 'Cases', 'Deaths']
)

(bar + tick0 + tick1 + tick2)

alt.LayerChart(...)